In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action = 'ignore')

In [2]:
amzn = pd.read_csv("dataset/AMZN_2006-01-01_to_2018-01-01.csv")
googl = pd.read_csv("dataset/GOOGL_2006-01-01_to_2018-01-01.csv")
ibm = pd.read_csv("dataset/IBM_2006-01-01_to_2018-01-01.csv")
msft = pd.read_csv("dataset/MSFT_2006-01-01_to_2018-01-01.csv")
aapl = pd.read_csv("dataset/AAPL_2006-01-01_to_2018-01-01.csv")
intc = pd.read_csv("dataset/INTC_2006-01-01_to_2018-01-01.csv")
vz = pd.read_csv("dataset/VZ_2006-01-01_to_2018-01-01.csv")
amd = pd.read_csv("dataset/AMD.csv")
nvda = pd.read_csv("dataset/nvidia_all_time_daily_stock_prices.csv")
aaba = pd.read_csv("dataset/AABA_2006-01-01_to_2018-01-01.csv")

In [3]:
amd['Date'] = pd.to_datetime(amd['Date'])
nvda['Date'] = pd.to_datetime(nvda['Date'])
start_date = pd.to_datetime('2006-01-03')
end_date = pd.to_datetime('2017-12-29')
amd_crop = amd[(amd['Date'] >= start_date) & (amd['Date'] <= end_date)]
amd_crop = amd_crop.reset_index(drop=True)
amd_crop['Close'] = amd_crop['Close'].round(2)
nvda_crop = nvda[(nvda['Date'] >= start_date) & (nvda['Date'] <= end_date)]
nvda_crop = nvda_crop.reset_index(drop = True)
nvda_crop['Close'] = nvda_crop['Close'].round(2)

In [76]:
data = pd.concat([amzn[['Date','Close']], googl["Close"], msft["Close"], aapl["Close"], intc['Close'], 
                  nvda_crop['Close'], ibm["Close"], vz['Close'], aaba['Close'], amd_crop['Close']], axis = 1)
data.columns = ['Date', 'amzn', 'googl', 'msft', 'aapl', 'intc','nvda', 'ibm', 'vz', 'aaba', 'amd']
data = data.drop(data.index[-1])
data['Date'] = pd.to_datetime(data['Date'])
cols = list(data)[1:11]
train_date = data['Date']
train_data = data[cols].astype(float)
np.shape(train_data)

(3019, 10)

In [77]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc = sc.fit(train_data)
scaled_train_data = sc.transform(train_data)
np.shape(scaled_train_data)

(3019, 10)

In [106]:
x_train = []
y_train = []

n_future = 1
n_past = 30

for i in range(n_past, len(scaled_train_data) - n_future + 1):
    x_train.append(scaled_train_data[i - n_past:i, 0: train_data.shape[1]])
    y_train.append(scaled_train_data[i :i + n_future, 0 : train_data.shape[1]])

x_train, y_train = np.array(x_train), np.array(y_train)

In [107]:
y_train[0]

array([[-0.92920627, -1.08624442, -0.65568105, -1.23004792, -0.60317184,
        -0.41593281, -1.72571852, -0.90881929,  0.34756226,  4.39697519]])

In [108]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam

In [109]:
model = Sequential()
model.add(LSTM(64, activation = 'relu', input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences = True))
model.add(LSTM(32, activation = 'relu', return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1]))
model.compile(optimizer = 'adam', loss = 'mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 30, 64)            19200     
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 31649 (123.63 KB)
Trainable params: 31649 (123.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [110]:
history = model.fit(x_train, y_train, epochs = 10, batch_size = 16, validation_split = 0.1, verbose = 1)

Epoch 1/10
169/169 [==============================] - 5s 14ms/step - loss: 0.6556 - val_loss: 3.8258
Epoch 2/10
169/169 [==============================] - 2s 12ms/step - loss: 0.6532 - val_loss: 3.6993
Epoch 3/10
169/169 [==============================] - 2s 12ms/step - loss: 0.6520 - val_loss: 3.5052
Epoch 4/10
169/169 [==============================] - 2s 12ms/step - loss: 0.6529 - val_loss: 3.8467
Epoch 5/10
169/169 [==============================] - 2s 12ms/step - loss: 0.6524 - val_loss: 3.5541
Epoch 6/10
169/169 [==============================] - 2s 12ms/step - loss: 0.6524 - val_loss: 4.1400
Epoch 7/10
169/169 [==============================] - 2s 12ms/step - loss: 0.6515 - val_loss: 3.6992
Epoch 8/10
169/169 [==============================] - 2s 13ms/step - loss: 0.6523 - val_loss: 4.1939
Epoch 9/10
169/169 [==============================] - 2s 12ms/step - loss: 0.6523 - val_loss: 3.7315
Epoch 10/10
169/169 [==============================] - 2s 12ms/step - loss: 0.6523 - val_lo